In [12]:
# advanced_time_series_full.py
# Full end-to-end: ARIMA (unscaled) + LSTM + Transformer (Keras) + metrics + plots
# Safe installer: installs missing packages into the same interpreter
# Author: (generated)
# Run in Jupyter or as a script. If running in Jupyter, run cell top->bottom.

import sys
import subprocess
import importlib
from packaging import version

def ensure(pkg_name, import_name=None):
    """Ensure package installed and importable. Returns imported module."""
    import_name = import_name or pkg_name
    try:
        return importlib.import_module(import_name)
    except Exception:
        print(f"Installing {pkg_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        return importlib.import_module(import_name)

# Required packages (minimal, stable set)
np = ensure("numpy")
pd = ensure("pandas")
plt = ensure("matplotlib.pyplot", "matplotlib.pyplot")
sns = ensure("seaborn")
statsmodels = ensure("statsmodels")
sklearn = ensure("scikit-learn")
tf = ensure("tensorflow")
joblib = ensure("joblib")
tqdm = ensure("tqdm")

# now import specific names safely
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras import layers, models, callbacks
import joblib
from tqdm import tqdm

import math
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print("Using device:", device)

Installing scikit-learn ...


ModuleNotFoundError: No module named 'scikit-learn'

In [ ]:

# -----------------------
# 1. Load (or create) data
# -----------------------
# If you have a CSV, replace this block with:
# df = pd.read_csv("yourfile.csv", parse_dates=['date_col'], index_col='date_col')
#
# For reproducibility if no file provided, synthetic series:
np.random.seed(42)
t = np.arange(0, 1000)
series = 0.001 * t + 2 * np.sin(2 * np.pi * t / 50) + 0.5 * np.sin(2 * np.pi * t / 7) + np.random.normal(0, 0.5, len(t))
df = pd.DataFrame({"value": series})
df.index = pd.date_range(start="2000-01-01", periods=len(df), freq="D")
print("Data sample:")
print(df.head())

In [2]:

# -----------------------
# 2. Stationarity tests
# -----------------------
def run_adf(x):
    res = adfuller(x, autolag='AIC')
    return {"ADF": res[0], "pvalue": res[1]}

def run_kpss(x):
    try:
        res = kpss(x, nlags='auto')
        return {"KPSS": res[0], "pvalue": res[1]}
    except Exception as e:
        return {"KPSS_error": str(e)}

print("\nADF test (first 1000 points):", run_adf(df['value']))
print("KPSS test:", run_kpss(df['value']))

NameError: name 'df' is not defined

In [3]:
# -----------------------
# 3. Train/test split & scaling
# -----------------------
H = 30               # forecast horizon
SEQ_LEN = 60         # lookback for LSTM/Transformer
train_frac = 0.8
n = len(df)
train_n = int(n * train_frac)

# ARIMA will be trained on raw unscaled train series (recommended)
train_series = df['value'].values[:train_n]
# For sequence models, we scale using scaler fitted only on train to avoid leakage
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df[['value']].iloc[:train_n].values.reshape(-1,1)).flatten()
# For convenience we'll also scale the rest using this scaler
scaled_all = scaler.transform(df[['value']].values.reshape(-1,1)).flatten()

# Create sequences (for multi-step H forecast)
def make_sequences(values, seq_len, horizon):
    X, y = [], []
    for i in range(len(values) - seq_len - horizon + 1):
        X.append(values[i:i+seq_len])
        y.append(values[i+seq_len:i+seq_len+horizon])
    return np.array(X), np.array(y)

X_all, y_all = make_sequences(scaled_all, SEQ_LEN, H)
# determine index splitting so training sequences end before train_n
# find last starting index that uses only train points (i + seq_len + horizon -1 < train_n)
last_train_start = train_n - seq_len - H + 1
if last_train_start < 1:
    raise ValueError("Not enough data for chosen SEQ_LEN/H. Reduce SEQ_LEN or H or increase data.")
train_idx = np.arange(0, last_train_start)
test_idx = np.arange(last_train_start, len(X_all))

X_train = X_all[train_idx]
y_train = y_all[train_idx]
X_test  = X_all[test_idx]
y_test  = y_all[test_idx]

print("\nShapes: X_train, y_train, X_test, y_test:", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# reshape for keras: (samples, seq_len, features)
X_train_keras = X_train.reshape((X_train.shape[0], SEQ_LEN, 1))
X_test_keras  = X_test.reshape((X_test.shape[0], SEQ_LEN, 1))


NameError: name 'df' is not defined

In [4]:
# -----------------------
# 4. ARIMA baseline (trained on unscaled original train)
# -----------------------
# Simple grid search for small p,d,q by AIC to avoid external auto_arima dependency
def select_arima_order(series, p_max=3, d_vals=[0,1], q_max=3):
    best_aic = np.inf
    best_order = None
    for p in range(p_max+1):
        for d in d_vals:
            for q in range(q_max+1):
                try:
                    model = ARIMA(series, order=(p,d,q))
                    res = model.fit(method_kwargs={"warn_convergence":False})
                    if res.aic < best_aic:
                        best_aic = res.aic
                        best_order = (p,d,q)
                except Exception:
                    continue
    return best_order

print("\nSelecting ARIMA order by small AIC search (this may take a few seconds)...")
arima_order = select_arima_order(train_series, p_max=3, d_vals=[0,1], q_max=3)
if arima_order is None:
    arima_order = (1,1,0)  # fallback
print("Chosen ARIMA order:", arima_order)

arima_model = ARIMA(train_series, order=arima_order)
arima_result = arima_model.fit()
# Forecast H steps from end of training period
arima_forecast = arima_result.forecast(steps=H)
arima_pred = np.array(arima_forecast).flatten()
print("ARIMA forecast (first 5):", arima_pred[:5])



Selecting ARIMA order by small AIC search (this may take a few seconds)...


NameError: name 'train_series' is not defined

In [5]:
# -----------------------
# 5. LSTM model (Keras)
# -----------------------
tf.keras.backend.clear_session()
lstm_units = 64
model_lstm = models.Sequential([
    layers.Input(shape=(SEQ_LEN, 1)),
    layers.LSTM(lstm_units, return_sequences=False),
    layers.Dense(H)
])
model_lstm.compile(optimizer='adam', loss='mse')
print("\nLSTM model summary:")
model_lstm.summary()

es = callbacks.EarlyStopping(monitor='loss', patience=6, restore_best_weights=True, min_delta=1e-6)
# Train (use modest epochs to keep runtime reasonable; increase if you want stronger fit)
EPOCHS_LSTM = 25
model_lstm.fit(X_train_keras, y_train.reshape((y_train.shape[0], H)), epochs=EPOCHS_LSTM, batch_size=32, callbacks=[es], verbose=1)

# Predict: take first test sequence forecast for direct comparability with ARIMA's single block
lstm_pred_scaled = model_lstm.predict(X_test_keras[:1])  # shape (1,H)
# inverse transform (use same scaler fitted on train)
lstm_pred = scaler.inverse_transform(lstm_pred_scaled.reshape(-1,1)).flatten()


NameError: name 'tf' is not defined

In [6]:
# -----------------------
# 6. Transformer model (Keras MultiHeadAttention)
# -----------------------
def build_transformer(seq_len, d_model=64, nhead=4, ff_dim=128, num_layers=1, horizon=H):
    inputs = layers.Input(shape=(seq_len, 1))
    x = layers.Dense(d_model)(inputs)  # project to d_model
    # positional embedding (learned)
    positions = tf.range(start=0, limit=seq_len, delta=1)
    pos_emb_layer = layers.Embedding(input_dim=seq_len, output_dim=d_model)
    pos_emb = pos_emb_layer(positions)
    x = x + pos_emb

    for _ in range(num_layers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn = layers.Dense(ff_dim, activation='relu')(x)
        ffn = layers.Dense(d_model)(ffn)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(horizon)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

tf.keras.backend.clear_session()
transformer_model = build_transformer(SEQ_LEN, d_model=64, nhead=4, ff_dim=128, num_layers=1, horizon=H)
transformer_model.compile(optimizer='adam', loss='mse')
print("\nTransformer model summary:")
transformer_model.summary()

EPOCHS_TRANS = 25
es2 = callbacks.EarlyStopping(monitor='loss', patience=6, restore_best_weights=True, min_delta=1e-6)
transformer_model.fit(X_train_keras, y_train.reshape((y_train.shape[0], H)), epochs=EPOCHS_TRANS, batch_size=32, callbacks=[es2], verbose=1)

trans_pred_scaled = transformer_model.predict(X_test_keras[:1])
trans_pred = scaler.inverse_transform(trans_pred_scaled.reshape(-1,1)).flatten()


NameError: name 'tf' is not defined

In [7]:
# -----------------------
# 7. Prepare true future block (first test example)
# -----------------------
y_true_block_scaled = y_test[0]  # scaled horizon for first test example
y_true_block = scaler.inverse_transform(y_true_block_scaled.reshape(-1,1)).flatten()


NameError: name 'y_test' is not defined

In [8]:
# -----------------------
# 8. Metrics: RMSE, MAE, Directional Accuracy
# -----------------------
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def directional_acc(y_true, y_pred):
    # handle small length case
    if len(y_true) < 2:
        return np.nan
    sign_true = np.sign(np.diff(y_true))
    sign_pred = np.sign(np.diff(y_pred))
    return np.mean(sign_true == sign_pred)

models_preds = {
    "ARIMA": arima_pred,
    "LSTM": lstm_pred,
    "Transformer": trans_pred
}

print("\n--- METRICS for first test horizon block ---")
for name, pred in models_preds.items():
    r = rmse(y_true_block, pred)
    m = mean_absolute_error(y_true_block, pred)
    d = directional_acc(y_true_block, pred)
    print(f"{name:12s} → RMSE: {r:.4f}, MAE: {m:.4f}, DirAcc: {d:.4f}")


NameError: name 'arima_pred' is not defined

In [9]:
# -----------------------
# 9. Plot results
# -----------------------
plt.figure(figsize=(12, 5))
plt.plot(y_true_block, label="True future (H)", marker='o')
plt.plot(arima_pred, label="ARIMA forecast", marker='o')
plt.plot(lstm_pred, label="LSTM forecast", marker='o')
plt.plot(trans_pred, label="Transformer forecast", marker='o')
plt.title("True vs ARIMA vs LSTM vs Transformer (first test block)")
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'y_true_block' is not defined

<Figure size 1200x500 with 0 Axes>

In [10]:
# -----------------------
# 10. Save models & scaler (optional)
# -----------------------
# Save Keras models and scaler/arima
model_lstm.save("model_lstm.h5")
transformer_model.save("model_transformer.h5")
joblib.dump(scaler, "scaler.gz")
arima_result.save("arima_result.pkl")  # statsmodels save

print("\nSaved files: model_lstm.h5, model_transformer.h5, scaler.gz, arima_result.pkl")

NameError: name 'model_lstm' is not defined

In [ ]:
# -----------------------------------------------------------
# FULL ADVANCED TIME SERIES PROJECT
# ARIMA + LSTM + TRANSFORMER (ERROR-FREE VERSION)
# -----------------------------------------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

import joblib

# -----------------------------------------------------------
# 1. Load Synthetic Dataset (Replace with your data)
# -----------------------------------------------------------
np.random.seed(42)
t = np.arange(0, 1000)
series = 0.001 * t + 2 * np.sin(2 * np.pi * t / 50) + \
         0.5 * np.sin(2 * np.pi * t / 7) + np.random.normal(0, 0.5, len(t))

df = pd.DataFrame({"value": series})
df.index = pd.date_range(start="2000-01-01", periods=len(df), freq="D")

print(df.head())

# -----------------------------------------------------------
# 2. Stationarity Tests
# -----------------------------------------------------------

def adf_test(series):
    res = adfuller(series)
    return {"ADF": res[0], "pvalue": res[1]}

def kpss_test(series):
    try:
        res = kpss(series, nlags="auto")
        return {"KPSS": res[0], "pvalue": res[1]}
    except:
        return {"KPSS": "ERROR"}

print("\nADF:", adf_test(df['value']))
print("KPSS:", kpss_test(df['value']))

# -----------------------------------------------------------
# 3. Train/Test Split and Scaling
# -----------------------------------------------------------
H = 30       # forecast horizon
SEQ_LEN = 60
train_ratio = 0.8

n = len(df)
train_n = int(n * train_ratio)

train_series = df['value'].values[:train_n]

scaler = MinMaxScaler()
scaled_all = scaler.fit_transform(df[['value']]).flatten()

# ---- Create Sequences ----
def create_sequences(values, seq_len, horizon):
    X, y = [], []
    for i in range(len(values) - seq_len - horizon + 1):
        X.append(values[i:i+seq_len])
        y.append(values[i+seq_len:i+seq_len+horizon])
    return np.array(X), np.array(y)

X_all, y_all = create_sequences(scaled_all, SEQ_LEN, H)

last_train_start = train_n - SEQ_LEN - H + 1
train_idx = np.arange(0, last_train_start)
test_idx = np.arange(last_train_start, len(X_all))

X_train = X_all[train_idx]
y_train = y_all[train_idx]
X_test  = X_all[test_idx]
y_test  = y_all[test_idx]

X_train = X_train.reshape((X_train.shape[0], SEQ_LEN, 1))
X_test  = X_test.reshape((X_test.shape[0], SEQ_LEN, 1))

# -----------------------------------------------------------
# 4. ARIMA (Unscaled Series)
# -----------------------------------------------------------

arima_model = ARIMA(train_series, order=(2,1,2))
arima_result = arima_model.fit()
arima_forecast = arima_result.forecast(steps=H)
arima_pred = np.array(arima_forecast)

# -----------------------------------------------------------
# 5. LSTM Model
# -----------------------------------------------------------

model_lstm = models.Sequential([
    layers.Input(shape=(SEQ_LEN,1)),
    layers.LSTM(64),
    layers.Dense(H)
])

model_lstm.compile(optimizer="adam", loss="mse")

model_lstm.fit(
    X_train, y_train,
    epochs=20, batch_size=32, verbose=1
)

lstm_pred_scaled = model_lstm.predict(X_test[:1])
lstm_pred = scaler.inverse_transform(lstm_pred_scaled.reshape(-1,1)).flatten()

# -----------------------------------------------------------
# 6. Transformer Model
# -----------------------------------------------------------

def build_transformer(seq_len, d_model=64, heads=4, ff_dim=128, horizon=H):
    inputs = layers.Input(shape=(seq_len,1))
    x = layers.Dense(d_model)(inputs)

    positions = tf.range(0, seq_len)
    pos_emb = layers.Embedding(input_dim=seq_len, output_dim=d_model)(positions)
    x = x + pos_emb

    attn = layers.MultiHeadAttention(num_heads=heads, key_dim=d_model)(x, x)
    x = layers.LayerNormalization()(x + attn)

    ffn = layers.Dense(ff_dim, activation="relu")(x)
    ffn = layers.Dense(d_model)(ffn)
    x = layers.LayerNormalization()(x + ffn)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(horizon)(x)

    return models.Model(inputs, outputs)

transformer = build_transformer(SEQ_LEN)
transformer.compile(optimizer="adam", loss="mse")

transformer.fit(
    X_train, y_train,
    epochs=20, batch_size=32, verbose=1
)

trans_pred_scaled = transformer.predict(X_test[:1])
trans_pred = scaler.inverse_transform(trans_pred_scaled.reshape(-1,1)).flatten()

# -----------------------------------------------------------
# 7. True Test Block
# -----------------------------------------------------------
y_true_scaled = y_test[0]
y_true = scaler.inverse_transform(y_true_scaled.reshape(-1,1)).flatten()

# -----------------------------------------------------------
# 8. Metrics
# -----------------------------------------------------------
def rmse(a,b): return np.sqrt(mean_squared_error(a,b))

print("\n--- METRICS ---")
print("ARIMA RMSE:", rmse(y_true, arima_pred))
print("LSTM RMSE :", rmse(y_true, lstm_pred))
print("TRANS RMSE:", rmse(y_true, trans_pred))

# -----------------------------------------------------------
# 9. Plot
# -----------------------------------------------------------
plt.figure(figsize=(14,6))
plt.plot(y_true, label="True")
plt.plot(arima_pred, label="ARIMA")
plt.plot(lstm_pred, label="LSTM")
plt.plot(trans_pred, label="Transformer")
plt.legend()
plt.grid()
plt.show()

# -----------------------------------------------------------
# 10. Save Models
# -----------------------------------------------------------
model_lstm.save("model_lstm.h5")
transformer.save("model_transformer.h5")
joblib.dump(scaler, "scaler.gz")
arima_result.save("arima_model.pkl")

print("Models saved.")
